# Financial information scraper

. 

### Import scraping tools

We can use a yahoo finance scraper that I found that I will post in the repo: github.com/lukaszbanasiak/yahoo-finance it can be built from source using the setup.py method (pip seemed to be broken). 

In [15]:
import yahoo_finance as yf

In [16]:
yahoo = yf.Share("YHOO")

In [17]:
print yahoo.get_open()

42.80
